In [1]:
import pandas as pd
from os import getcwd, listdir
from os.path import join, isfile, exists, abspath, pardir
from sys import path
import re

##### Configs

In [2]:
database = "sdm"
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")

In [3]:
path.append(scripts_dir)
from connect import Neo4jConnection

In [4]:
driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

##### Helper methods

In [5]:
def run_query(query):
    """
    Basic wrapper around 'driver' object
    """
    return driver.query(query)

#### Queries

In [6]:
def most_cited_papers():
    query = """
        Match (d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper' and toInteger(d.cited_count) > 0
        With j.name as conference, d.title as paper, sum(toInteger(d.cited_count)) as cited_count
        order by cited_count desc
        return conference, collect(paper)[..3] as papers, collect(cited_count)[..3] as total_cited_count
    """
    run_query(query=query)

In [7]:
most_cited_papers()

In [6]:
def community():
    query = """
        MATCH (a:Author)<-[p:written_by]-(d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper'
        with j.name as conference, a.author_id as author_id, a.name as author_name, count(p) as count_paper
        order by author_name
        where count_paper > 3
        return conference, collect(author_name) as authors, sum(count_paper) as total_count
    """
    run_query(query=query)

In [7]:
community()